## Installation

In [32]:
%pip install -q pyomo
!apt-get install coinor-cbc

Note: you may need to restart the kernel to use updated packages.
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


#### Initializing libraries and model

In [33]:
from pyomo.environ import *
import math
model=ConcreteModel()
model.constraints=ConstraintList()

# Modelling

Add initial variables here, like importing the game files, the rules, etc, which are constants,or decided by the game

In [34]:
n=50                              #Number of match ticks
E=[1 for i in range (n)]            #Enemy strength for each attack tick
T=1000                               #Treasure required to reach victory
V=[i for i in range(n)]             #Number of jobs in each match tick
Cg=100                              #Amount of gold one can generate in each job for one job tick
Cf=100                              #Amount of food one can generate in each job for one job tick
Ct=100                              #Amount of treasure one can generate in each job for one job tick
Tg=20                               #Amount of gold required to train one soldier
Tf=50                               #Amount of food required to train one soldier
M=10000000                           #The big M we will be using
Wininc=100000                       #The army winning increase, needs to be optimized for better solution
Str_Sol=72                          #Strength of one soldier
armyV_initial=10                    #Initialized as needed, the initial value of army

#### Instruction for modelling
Add variables by ``model.x=Var(range(_dimension_),domain=[_the domain of the variable_])``

In [35]:
model.food_in=Var(range(n), domain=NonNegativeIntegers)                 #Number of jobs allocated to food in each tick
model.gold_in=Var(range(n),domain=NonNegativeIntegers)                  #Number of jobs allocated to gold in each tick
model.treasure_in=Var(range(n),domain=NonNegativeIntegers)              #Number of jobs allocated to treasure in each tick
model.total_gold=Var(range(n),domain=NonNegativeIntegers)               #Represents gold before ith attack {deducting all costs}
model.total_food=Var(range(n),domain=NonNegativeIntegers)               #Represents food before ith attack {deducting all costs}
model.total_treasure=Var(range(n),domain=NonNegativeIntegers)           #Represents treasure before ith attack {deducting all costs}
model.train=Var(range(n),domain=NonNegativeIntegers)                    #Represents the number of soldiers trained before ith attack
model.army=Var(range(n),domain=NonNegativeIntegers)                     #Represents the strength of total army before ith attack
model.armyV=Var(range(n),domain=NonNegativeIntegers)                    #Represents the strength of total army after ith attack
model.W=Var(range(n),domain=Binary)                                     #Represents binary which denotes whether won at or before ith attack or not
model.r=Var(range(n),domain=[1.0000,0.3333,0.6667,0.5000,0.2000,0.8000,0.0000,0.5714])  #Represents the  fraction of army which should survive
model.x1_25=Var(range(n),domain=Binary)                                 #The next few variables sets rule for wars, the damage our army gets in attack
model.x1_5=Var(range(n),domain=Binary)
model.x1_75=Var(range(n),domain=Binary)
model.x2=Var(range(n),domain=Binary)
model.x3=Var(range(n),domain=Binary)
model.x5=Var(range(n),domain=Binary)

Objective function

In [36]:
model.obj = Objective(expr = n-sum(model.W[i] for i in range (n)), sense = minimize)          #Gives the fastest possible win, collecting T amounts of treasure

Add constraints by ``model.constraint.add(expr=...)``

In [37]:
for i in range(1,n):
    model.constraints.add(expr=model.total_gold[i]==model.total_gold[i-1]+Cg*model.gold_in[i]-Tg*model.train[i])
    model.constraints.add(expr=model.total_food[i]==model.total_food[i-1]+Cf*model.food_in[i]-Tf*model.train[i])
    model.constraints.add(expr=model.total_treasure[i]==model.total_treasure[i-1]+Ct*model.treasure_in[i])
    model.constraints.add(expr=model.army[i]==model.armyV[i-1]+Str_Sol*model.train[i]+Wininc*model.W[i])
    model.constraints.add(expr=model.armyV[i]>=0)
    model.constraints.add(expr=model.army[i]>=E[i])
    model.constraints.add(expr=model.armyV[i]==model.army[i]-E[i]*model.r[i])
    #Now comes warzone constraints
    model.constraints.add(expr=model.army[i]-1.25*E[i]-1<=M*model.x1_25[i])           #x_{1.25}
    model.constraints.add(expr=model.army[i]-1.25*E[i]-1>=-M*(1-model.x1_25[i]))
    model.constraints.add(expr=model.r[i]-0.8000<=M*(1-model.x1_25[i]))
    model.constraints.add(expr=model.r[i]-0.8000>=-M*(model.x1_25[i]))

    model.constraints.add(expr=model.army[i]-1.5*E[i]-1<=M*model.x1_5[i])             #x_{1.5}
    model.constraints.add(expr=model.army[i]-1.5*E[i]-1>=-M*(1-model.x1_5[i]))
    model.constraints.add(expr=model.r[i]-0.6667<=M*(1-model.x1_5[i]))
    model.constraints.add(expr=model.r[i]-0.6667>=-M*(model.x1_5[i]))

    model.constraints.add(expr=model.army[i]-1.75*E[i]-1<=M*model.x1_75[i])           #x_{1.75}
    model.constraints.add(expr=model.army[i]-1.75*E[i]-1>=-M*(1-model.x1_75[i]))
    model.constraints.add(expr=model.r[i]-0.5714<=M*(1-model.x1_5[i]))
    model.constraints.add(expr=model.r[i]-0.5714>=-M*(model.x1_5[i]))

    model.constraints.add(expr=model.army[i]-2*E[i]-1<=M*model.x2[i])                 #x_{2}
    model.constraints.add(expr=model.army[i]-2*E[i]-1>=-M*(1-model.x2[i]))
    model.constraints.add(expr=model.r[i]-0.5000<=M*(1-model.x2[i]))
    model.constraints.add(expr=model.r[i]-0.5000>=-M*(model.x2[i]))

    model.constraints.add(expr=model.army[i]-3*E[i]-1<=M*model.x3[i])                 #x_{3}
    model.constraints.add(expr=model.army[i]-3*E[i]-1>=-M*(1-model.x3[i]))
    model.constraints.add(expr=model.r[i]-0.3333<=M*(1-model.x3[i]))
    model.constraints.add(expr=model.r[i]-0.3333>=-M*(model.x3[i]))

    model.constraints.add(expr=model.army[i]-5*E[i]-1<=M*model.x5[i])                 #x_{5}
    model.constraints.add(expr=model.army[i]-5*E[i]-1>=-M*(1-model.x5[i]))
    model.constraints.add(expr=model.r[i]-0.2000<=M*(1-model.x5[i]))
    model.constraints.add(expr=model.r[i]-0.2000>=-M*(model.x5[i]))

    model.constraints.add(expr=model.r[i]<=1-model.x5[i])

model.constraints.add(expr=model.total_treasure[0]==0)
model.constraints.add(expr=model.armyV[0]==armyV_initial)

for i in range(n):
    model.constraints.add(expr=model.total_gold[i]>=0)
    model.constraints.add(expr=model.total_food[i]>=0)
    model.constraints.add(expr=model.food_in[i]+model.gold_in[i]+model.treasure_in[i]==V[i])
    model.constraints.add(expr=model.total_treasure[i]<=M*model.W[i]+T-1)
    model.constraints.add(expr=model.total_treasure[i]>=-M*(1-model.W[i])+T-1)
    model.constraints.add(expr=model.r[i]<=1)
    model.constraints.add(expr=model.r[i]>=0)
    model.constraints.add(expr=model.W[i]>=0)
    model.constraints.add(expr=model.W[i]<=1)

## Solver using CBC

In [38]:
opt = SolverFactory('cbc')
result=opt.solve(model,tee=True)
print('Solver status: ',result.solver.status)
print('Solver termination condition: ',result.solver.termination_condition)

Welcome to the CBC MILP Solver 
Version: 2.10.8 
Build Date: May  9 2022 

command line - /bin/cbc -printingOptions all -import /tmp/tmpf8bsv5wj.pyomo.lp -stat=1 -solve -solu /tmp/tmpf8bsv5wj.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve is modifying 56 integer bounds and re-presolving
Presolve 1444 (-576) rows, 756 (-87) columns and 3381 (-701) elements
Statistics for presolved model
Original problem has 792 integers (344 of which binary)
Presolved problem has 707 integers (343 of which binary)
==== 707 zero objective 2 different
49 variables have objective of -1
707 variables have objective of 0
==== absolute objective values 2 different
707 variables have objective of 0
49 variables have objective of 1
==== for integers 658 zero objective 2 different
49 variables have objective of -1
658 variables have objective of 0
==== for integers absolute objective values 2 different
658 variables have objective of 0
49 variables have objective o

KeyboardInterrupt: 

In [ ]:
print('Optimal N',model.obj())

Optimal N 4.0


#### Points to ponder
Total food,gold initial, E[0] should be 0 or smh